<a href="https://colab.research.google.com/github/SarahKashef/blog/blob/master/FlowNet2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup and Install FlowNet2

## Download compatible Torch

In [2]:
!pip install torch==1.0.0 torchvision==0.2.2 -f https://download.pytorch.org/whl/cu90/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu90/torch_stable.html
     |████████████████████████████████| 591.8 MB 653 bytes/s 
     |████████████████████████████████| 64 kB 2.6 MB/s 
     |████████████████████████████████| 52 kB 1.3 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.62.3
    Uninstalling tqdm-4.62.3:
      Successfully uninstalled tqdm-4.62.3
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.0.0 wh

## Download and setup FlowNet2 files

In [3]:
import os
# get flownet2-pytorch source
!git clone https://github.com/Gauravv97/flownet2-pytorch.git
!mv /content/flownet2-pytorch /content/flownet2pytorch
os.chdir('./flownet2pytorch')
# install custom layers
!bash install.sh

Cloning into 'flownet2-pytorch'...
remote: Enumerating objects: 572, done.
remote: Total 572 (delta 0), reused 0 (delta 0), pack-reused 572
Receiving objects: 100% (572/572), 6.28 MiB | 16.37 MiB/s, done.
Resolving deltas: 100% (323/323), done.
mv: cannot stat '/content/flownet2-pytorch': No such file or directory


FileNotFoundError: ignored

### Add packages to IPython system path

In [4]:
import os
os.sys.path.append('/root/.local/lib/python3.6/site-packages/resample2d_cuda-0.0.0-py3.6-linux-x86_64.egg')
os.sys.path.append('/root/.local/lib/python3.6/site-packages/correlation_cuda-0.0.0-py3.6-linux-x86_64.egg')
os.sys.path.append( '/root/.local/lib/python3.6/site-packages/channelnorm_cuda-0.0.0-py3.6-linux-x86_64.egg')

# Download files and Install Packages

In [5]:
!pip install pypng
!pip install tensorboardx
!pip install  setproctitle colorama scipy==1.1.0

     |████████████████████████████████| 48 kB 2.4 MB/s 
     |████████████████████████████████| 124 kB 5.4 MB/s 
     |████████████████████████████████| 31.2 MB 1.4 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc3 3.11.4 requires scipy>=1.2.0, but you have scipy 1.1.0 which is incompatible.
plotnine 0.6.0 requires scipy>=1.2.0, but you have scipy 1.1.0 which is incompatible.
jax 0.2.25 requires scipy>=1.2.1, but you have scipy 1.1.0 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [6]:
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1hF8vS6YeHkx3j2pfCeQqqZGwA_PJq_Da',dest_path='./FlowNet2_checkpoint.pth.tar')

### Install scipy as some tensorflow functionality requires updated scipy

In [7]:
!pip install scipy==1.4.1

     |████████████████████████████████| 26.1 MB 1.7 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.1.0
    Uninstalling scipy-1.1.0:
      Successfully uninstalled scipy-1.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [8]:
os.chdir('/content/flownet2pytorch')

### Define show_flow() for visualization.
 Original Source https://github.com/sampepose/flownet2-tf/blob/master/src/flowlib.py

In [9]:
# Source:https://github.com/sampepose/flownet2-tf/blob/master/src/flowlib.py
import matplotlib.pyplot as plt
import numpy as np

UNKNOWN_FLOW_THRESH = 1e7
def show_flow(filename):
    """
    visualize optical flow map using matplotlib
    :param filename: optical flow file
    :return: None
    """
    flow = read_flow(filename)
    img = flow_to_image(flow)
    plt.imshow(img)
    plt.show()

def read_flow(filename):
    """
    read optical flow from Middlebury .flo file
    :param filename: name of the flow file
    :return: optical flow data in matrix
    """
    f = open(filename, 'rb')
    magic = np.fromfile(f, np.float32, count=1)
    data2d = None

    if 202021.25 != magic:
        print ('Magic number incorrect. Invalid .flo file')
    else:
        w = int(np.fromfile(f, np.int32, count=1)[0])
        h = int(np.fromfile(f, np.int32, count=1)[0])
        #print("Reading %d x %d flo file" % (h, w))
        data2d = np.fromfile(f, np.float32, count=2 * w * h)
        # reshape data into 3D array (columns, rows, channels)
        data2d = np.resize(data2d, (h, w, 2))
    f.close()
    return data2d

def flow_to_image(flow):
    """
    Convert flow into middlebury color code image
    :param flow: optical flow map
    :return: optical flow image in middlebury color
    """
    u = flow[:, :, 0]
    v = flow[:, :, 1]

    maxu = -999.
    maxv = -999.
    minu = 999.
    minv = 999.

    idxUnknow = (abs(u) > UNKNOWN_FLOW_THRESH) | (abs(v) > UNKNOWN_FLOW_THRESH)
    u[idxUnknow] = 0
    v[idxUnknow] = 0

    maxu = max(maxu, np.max(u))
    minu = min(minu, np.min(u))

    maxv = max(maxv, np.max(v))
    minv = min(minv, np.min(v))

    rad = np.sqrt(u ** 2 + v ** 2)
    maxrad = max(-1, np.max(rad))

    #print( "max flow: %.4f\nflow range:\nu = %.3f .. %.3f\nv = %.3f .. %.3f" % (maxrad, minu,maxu, minv, maxv))

    u = u/(maxrad + np.finfo(float).eps)
    v = v/(maxrad + np.finfo(float).eps)

    img = compute_color(u, v)

    idx = np.repeat(idxUnknow[:, :, np.newaxis], 3, axis=2)
    img[idx] = 0

    return np.uint8(img)


def compute_color(u, v):
    """
    compute optical flow color map
    :param u: optical flow horizontal map
    :param v: optical flow vertical map
    :return: optical flow in color code
    """
    [h, w] = u.shape
    img = np.zeros([h, w, 3])
    nanIdx = np.isnan(u) | np.isnan(v)
    u[nanIdx] = 0
    v[nanIdx] = 0

    colorwheel = make_color_wheel()
    ncols = np.size(colorwheel, 0)

    rad = np.sqrt(u**2+v**2)

    a = np.arctan2(-v, -u) / np.pi

    fk = (a+1) / 2 * (ncols - 1) + 1

    k0 = np.floor(fk).astype(int)

    k1 = k0 + 1
    k1[k1 == ncols+1] = 1
    f = fk - k0

    for i in range(0, np.size(colorwheel,1)):
        tmp = colorwheel[:, i]
        col0 = tmp[k0-1] / 255
        col1 = tmp[k1-1] / 255
        col = (1-f) * col0 + f * col1

        idx = rad <= 1
        col[idx] = 1-rad[idx]*(1-col[idx])
        notidx = np.logical_not(idx)

        col[notidx] *= 0.75
        img[:, :, i] = np.uint8(np.floor(255 * col*(1-nanIdx)))

    return img


def make_color_wheel():
    """
    Generate color wheel according Middlebury color code
    :return: Color wheel
    """
    RY = 15
    YG = 6
    GC = 4
    CB = 11
    BM = 13
    MR = 6

    ncols = RY + YG + GC + CB + BM + MR

    colorwheel = np.zeros([ncols, 3])

    col = 0

    # RY
    colorwheel[0:RY, 0] = 255
    colorwheel[0:RY, 1] = np.transpose(np.floor(255*np.arange(0, RY) / RY))
    col += RY

    # YG
    colorwheel[col:col+YG, 0] = 255 - np.transpose(np.floor(255*np.arange(0, YG) / YG))
    colorwheel[col:col+YG, 1] = 255
    col += YG

    # GC
    colorwheel[col:col+GC, 1] = 255
    colorwheel[col:col+GC, 2] = np.transpose(np.floor(255*np.arange(0, GC) / GC))
    col += GC

    # CB
    colorwheel[col:col+CB, 1] = 255 - np.transpose(np.floor(255*np.arange(0, CB) / CB))
    colorwheel[col:col+CB, 2] = 255
    col += CB

    # BM
    colorwheel[col:col+BM, 2] = 255
    colorwheel[col:col+BM, 0] = np.transpose(np.floor(255*np.arange(0, BM) / BM))
    col += + BM

    # MR
    colorwheel[col:col+MR, 2] = 255 - np.transpose(np.floor(255 * np.arange(0, MR) / MR))
    colorwheel[col:col+MR, 0] = 255

    return colorwheel

# Download Dataset from google drive

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1k5c4nJzmKkS55vg1K61o8Z_rr0FiMJow',dest_path='./480p')

## Convert to .png

In [ ]:
# from PIL import Image
# from os import listdir
# from os.path import splitext

# target_directory = "/content/sample_data/ftgyhuji"
# target = '.png'

# for file in listdir(target_directory):
#     filename, extension = splitext(file)
#     try:
#         if extension not in [target]:
#             im = Image.open(filename + extension)
#             im.save(filename + target)
#     except OSError:
#         print('Cannot convert %s' % file)

Cannot convert 00002.jpg
Cannot convert 00009.jpg
Cannot convert 00013.jpg
Cannot convert 00020.jpg
Cannot convert 00004.jpg
Cannot convert 00012.jpg
Cannot convert 00001.jpg
Cannot convert 00000.jpg
Cannot convert 00022.jpg
Cannot convert 00008.jpg
Cannot convert 00014.jpg
Cannot convert 00007.jpg
Cannot convert 00015.jpg
Cannot convert 00019.jpg
Cannot convert 00003.jpg
Cannot convert 00025.jpg
Cannot convert 00024.jpg
Cannot convert 00006.jpg
Cannot convert 00010.jpg
Cannot convert 00016.jpg
Cannot convert 00027.jpg
Cannot convert 00005.jpg
Cannot convert 00017.jpg
Cannot convert 00023.jpg
Cannot convert 00026.jpg
Cannot convert 00021.jpg
Cannot convert 00018.jpg
Cannot convert 00011.jpg
Cannot convert 00028.jpg
Cannot convert 00029.jpg


# Moving Frames Directory

In [ ]:
import os
from PIL import Image

directory = "/content/sample_data"

for root, subdirectories, files in os.walk(directory):
    for subdirectory in subdirectories:
        print(os.path.join(root, subdirectory))
    for file in files:
        print(os.path.join(root, file))
        

/content/sample_data/ftgyhuji
/content/sample_data/.ipynb_checkpoints
/content/sample_data/anscombe.json
/content/sample_data/README.md
/content/sample_data/california_housing_train.csv
/content/sample_data/california_housing_test.csv
/content/sample_data/mnist_test.csv
/content/sample_data/mnist_train_small.csv
/content/sample_data/ftgyhuji/00002.jpg
/content/sample_data/ftgyhuji/00009.jpg
/content/sample_data/ftgyhuji/00013.jpg
/content/sample_data/ftgyhuji/00020.jpg
/content/sample_data/ftgyhuji/00004.jpg
/content/sample_data/ftgyhuji/00012.jpg
/content/sample_data/ftgyhuji/00001.jpg
/content/sample_data/ftgyhuji/00000.jpg
/content/sample_data/ftgyhuji/00022.jpg
/content/sample_data/ftgyhuji/00008.jpg
/content/sample_data/ftgyhuji/00014.jpg
/content/sample_data/ftgyhuji/00007.jpg
/content/sample_data/ftgyhuji/00015.jpg
/content/sample_data/ftgyhuji/00019.jpg
/content/sample_data/ftgyhuji/00003.jpg
/content/sample_data/ftgyhuji/00025.jpg
/content/sample_data/ftgyhuji/00024.jpg
/conte

# Run the inference

### Generate .flo files using FlowNet2

In [7]:
!python main.py --inference --model FlowNet2 --save_flow --save ./output --inference_dataset ImagesFromFolder --inference_dataset_root ./frames/ --resume ./FlowNet2_checkpoint.pth.tar

Traceback (most recent call last):
  File "/content/flownet2pytorch/models.py", line 9, in <module>
    from networks.resample2d_package.resample2d import Resample2d
  File "/content/flownet2pytorch/networks/resample2d_package/resample2d.py", line 3, in <module>
    import resample2d_cuda
ModuleNotFoundError: No module named 'resample2d_cuda'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "main.py", line 16, in <module>
    import models, losses, datasets
  File "/content/flownet2pytorch/models.py", line 19, in <module>
    from .networks.resample2d_package.resample2d import Resample2d
ImportError: attempted relative import with no known parent package


# Visualizing flo files

### Save Flo files as images

In [11]:
import os
import PIL.Image
def mkdir_ifnotexists(dir):
    if os.path.exists(dir):
        return
    os.mkdir(dir)


flo_pth='/content/flownet2pytorch/output/inference/run.epoch-0-flow-field/'
flos=[flo_pth + f for f in os.listdir(flo_pth)]
mkdir_ifnotexists('./FlowFrames')
for i in range(len(flos)):
 PIL.Image.fromarray(flow_to_image(read_flow(flos[i]))).save('./FlowFrames/'+os.path.basename(flos[i])+'.png')

In [12]:
! zip FlowFrames.zip './FlowFrames/'*.png

  adding: FlowFrames/000000.flo.png (deflated 0%)
  adding: FlowFrames/000001.flo.png (deflated 0%)
  adding: FlowFrames/000002.flo.png (deflated 0%)
  adding: FlowFrames/000003.flo.png (deflated 0%)
  adding: FlowFrames/000004.flo.png (deflated 0%)
  adding: FlowFrames/000005.flo.png (deflated 0%)
  adding: FlowFrames/000006.flo.png (deflated 0%)
  adding: FlowFrames/000007.flo.png (deflated 0%)
  adding: FlowFrames/000008.flo.png (deflated 0%)
  adding: FlowFrames/000009.flo.png (deflated 0%)
  adding: FlowFrames/000010.flo.png (deflated 0%)
  adding: FlowFrames/000011.flo.png (deflated 0%)
  adding: FlowFrames/000012.flo.png (deflated 0%)
  adding: FlowFrames/000013.flo.png (deflated 0%)
  adding: FlowFrames/000014.flo.png (deflated 0%)
  adding: FlowFrames/000015.flo.png (deflated 0%)
  adding: FlowFrames/000016.flo.png (deflated 0%)
  adding: FlowFrames/000017.flo.png (deflated 0%)
  adding: FlowFrames/000018.flo.png (deflated 0%)
  adding: FlowFrames/000019.flo.png (deflated 0%)
